In [ ]:
import pandas as pd

# Load the datasets
real_news = pd.read_csv('True.csv')

# Printing head and tail
print(real_news)

# Print length => rows
print("Length (rows):", len(real_news))


                                                   title  \
0      As U.S. budget fight looms, Republicans flip t...   
1      U.S. military to accept transgender recruits o...   
2      Senior U.S. Republican senator: 'Let Mr. Muell...   
3      FBI Russia probe helped by Australian diplomat...   
4      Trump wants Postal Service to charge 'much mor...   
...                                                  ...   
21412  'Fully committed' NATO backs new U.S. approach...   
21413  LexisNexis withdrew two products from Chinese ...   
21414  Minsk cultural hub becomes haven from authorities   
21415  Vatican upbeat on possibility of Pope Francis ...   
21416  Indonesia to buy $1.14 billion worth of Russia...   

                                                    text       subject  \
0      WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1      WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2      WASHINGTON (Reuters) - The special counsel inv... 

In [ ]:
import pandas as pd

# Load the datasets
fake_news = pd.read_csv('Fake.csv')

# Printing head and tail
print(fake_news)

# Print length => rows
print("Length (rows):", len(fake_news))


                                                   title  \
0       Donald Trump Sends Out Embarrassing New Year’...   
1       Drunk Bragging Trump Staffer Started Russian ...   
2       Sheriff David Clarke Becomes An Internet Joke...   
3       Trump Is So Obsessed He Even Has Obama’s Name...   
4       Pope Francis Just Called Out Donald Trump Dur...   
...                                                  ...   
23476  McPain: John McCain Furious That Iran Treated ...   
23477  JUSTICE? Yahoo Settles E-mail Privacy Class-ac...   
23478  Sunnistan: US and Allied ‘Safe Zone’ Plan to T...   
23479  How to Blow $700 Million: Al Jazeera America F...   
23480  10 U.S. Navy Sailors Held by Iranian Military ...   

                                                    text      subject  \
0      Donald Trump just couldn t wish all Americans ...         News   
1      House Intelligence Committee Chairman Devin Nu...         News   
2      On Friday, it was revealed that former Milwauk...    

In [ ]:
# Load the dataset
fake_news = pd.read_csv('Fake.csv')
real_news = pd.read_csv('True.csv')

# Combine the datasets and shuffle
fake_news['label'] = 0   
real_news['label'] = 1

data = pd.concat([fake_news, real_news]).sample(frac=1.0)   # frac 1.0 => returns all the data (rows)

# Write the combined data to a new CSV file
data.to_csv('Combined.csv', index=False)

print("Data loaded successfully and combined CSV generated!")

data

Data loaded successfully and combined CSV generated!


,title,text,subject,date,label
11777,UK's May to visit China around Jan. 31: Sky News,(Reuters) - British Prime Minister Theresa May...,worldnews,"December 20, 2017",1
2124,Trump Decides That Starvation Should Be A Cen...,Programs that have kept hungry Americans fed f...,News,"March 16, 2017",0
23270,"SYRIA CEASEFIRE? Lavrov, Kerry Agree to Fight ...",21st Century Wire says After a long drawn out ...,Middle-east,"September 10, 2016",0
11660,Indonesian police warn Islamists against raids...,JAKARTA (Reuters) - Indonesian police appealed...,worldnews,"December 21, 2017",1
16319,Hot air? U.S. gas exporters rush to sell LNG t...,BEIJING (Reuters) - U.S. gas exporters and tra...,worldnews,"October 27, 2017",1
...,...,...,...,...,...
21160,CHER ATTACKS TRUMP: Compares Bill Clinton’s In...,"Really Cher? We re not excusing infidelity, bu...",left-news,"Jan 6, 2016",0
8472,Clinton leads Trump by five points in White Ho...,NEW YORK (Reuters) - Democrat Hillary Clinton ...,politicsNews,"August 12, 2016",1
663,Pathetic: Trump Actually Needs A General To B...,With Reince Priebus out as White House chief o...,News,"August 5, 2017",0
12190,OBAMA TELLS TROOPS To Rise Up Against Trump…Pr...,43 days and counting Characterizing the milita...,politics,"Dec 8, 2016",0


In [ ]:
import re
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

def preprocess_text(text):
    # Replace characters that are not between a to z or A to Z with whitespace
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Convert all characters into lowercase
    text = text.lower()

    # Remove inflectional morphemes like "ed", "est", "s", and "ing" from their token stem
    text = [stemmer.stem(word) for word in text.split()]

    # Join the processed words back into a single string
    text = ' '.join(text)

    return text


In [6]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
print("GPUs Available:", tf.config.list_physical_devices('GPU'))


# Check if GPU is available
if tf.config.list_physical_devices('GPU'):
    device_name = 'GPU'
else:
    device_name = 'CPU'
print('Using device:', device_name)


# Load the dataset
data = pd.read_csv('Combined.csv')

# Preprocess the text using the preprocessing function
print("\nPreprocessing data...")

data['title_preprocessed'] = data['title'].apply(preprocess_text)

# Split the dataset into train, validation, and test sets
train_ratio = 0.64
val_ratio = 0.16
test_ratio = 0.2

print("\nSplitting Data...")

train_data = data.sample(frac=train_ratio, random_state=42)
remaining_data = data.drop(train_data.index)

val_data = remaining_data.sample(frac=val_ratio/(val_ratio+test_ratio), random_state=42)
test_data = remaining_data.drop(val_data.index)

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the titles and convert them into BERT input tensors
print("\nTokenizing and Converting...")

train_inputs = tokenizer(list(train_data['title_preprocessed']), truncation=True, padding='max_length', max_length=42, return_tensors='tf')
val_inputs = tokenizer(list(val_data['title_preprocessed']), truncation=True, padding='max_length', max_length=42, return_tensors='tf')
test_inputs = tokenizer(list(test_data['title_preprocessed']), truncation=True, padding='max_length', max_length=42, return_tensors='tf')

# Convert the labels into TensorFlow tensors
train_labels = tf.convert_to_tensor(list(train_data['label']))
val_labels = tf.convert_to_tensor(list(val_data['label']))
test_labels = tf.convert_to_tensor(list(test_data['label']))

# Extract token tensors, segment tensors, and mask tensors from the BERT inputs
train_token_tensors = train_inputs['input_ids']
train_segment_tensors = train_inputs['token_type_ids']
train_mask_tensors = train_inputs['attention_mask']

val_token_tensors = val_inputs['input_ids']
val_segment_tensors = val_inputs['token_type_ids']
val_mask_tensors = val_inputs['attention_mask']

test_token_tensors = test_inputs['input_ids']
test_segment_tensors = test_inputs['token_type_ids']
test_mask_tensors = test_inputs['attention_mask']

# Build the BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Compile the model
print("\nCompiling Model...")

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train the BERT model
print("\nTraining Model...")

batch_size = 64
num_epochs = 15

history = model.fit([train_token_tensors, train_segment_tensors, train_mask_tensors], train_labels, batch_size=batch_size, epochs=num_epochs, validation_data=([val_token_tensors, val_segment_tensors, val_mask_tensors], val_labels))

# Save the trained model
print("\nSaving Model...")

model.save_pretrained('/content/bertv3_model')



GPUs Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Using device: GPU

Preprocessing data...

Splitting Data...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]


Tokenizing and Converting...


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Compiling Model...

Training Model...
Epoch 1/15
449/449 [==============================] - 297s 553ms/step - loss: 0.1667 - accuracy: 0.9381 - val_loss: 0.0861 - val_accuracy: 0.9687
Epoch 2/15
449/449 [==============================] - 247s 551ms/step - loss: 0.0746 - accuracy: 0.9741 - val_loss: 0.0860 - val_accuracy: 0.9694
Epoch 3/15
449/449 [==============================] - 248s 553ms/step - loss: 0.0383 - accuracy: 0.9865 - val_loss: 0.0939 - val_accuracy: 0.9715
Epoch 4/15
449/449 [==============================] - 248s 553ms/step - loss: 0.0242 - accuracy: 0.9921 - val_loss: 0.0866 - val_accuracy: 0.9766
Epoch 5/15
449/449 [==============================] - 248s 553ms/step - loss: 0.0130 - accuracy: 0.9957 - val_loss: 0.1130 - val_accuracy: 0.9766
Epoch 6/15
449/449 [==============================] - 248s 553ms/step - loss: 0.0116 - accuracy: 0.9961 - val_loss: 0.1062 - val_accuracy: 0.9751
Epoch 7/15
449/449 [==============================] - 247s 550ms/step - loss: 0.0093 

In [7]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate([test_token_tensors, test_segment_tensors, test_mask_tensors], test_labels, batch_size=batch_size)
print(f'Test loss: {loss * 100:.3f}%')
print(f'Test accuracy: {accuracy * 100:.3f}%')

# Import the necessary metrics
from sklearn.metrics import classification_report

# Get predictions on the test set
y_pred_logits = model.predict([test_token_tensors, test_segment_tensors, test_mask_tensors]).logits
y_pred = tf.argmax(tf.nn.softmax(y_pred_logits), axis=1).numpy()
y_true = test_labels.numpy()

# Generate classification report
report = classification_report(y_true, y_pred, target_names=['Fake', 'Real'])

# Print classification report without repeating accuracy
print("\nClassification Report:\n", report)


141/141 [==============================] - 27s 193ms/step - loss: 0.1599 - accuracy: 0.9737
Test loss: 15.988%
Test accuracy: 97.372%
281/281 [==============================] - 29s 94ms/step

Classification Report:
               precision    recall  f1-score   support

        Fake       0.98      0.97      0.97      4695
        Real       0.97      0.97      0.97      4284

    accuracy                           0.97      8979
   macro avg       0.97      0.97      0.97      8979
weighted avg       0.97      0.97      0.97      8979



In [9]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

# Load the saved model
model = TFBertForSequenceClassification.from_pretrained('/content/bertv3_model')

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example input text
input_text = input("\n\nEnter News Title: ")

# Preprocess the input text
preprocessed_text = preprocess_text(input_text)

print("Preprocessed Text:", preprocessed_text)

# Tokenize the preprocessed text
inputs = tokenizer(preprocessed_text, truncation=True, padding='max_length', max_length=42, return_tensors='tf')

# Extract input tensors
token_tensors = inputs['input_ids']
segment_tensors = inputs['token_type_ids']
mask_tensors = inputs['attention_mask']

# Make predictions
predictions = model.predict([token_tensors, segment_tensors, mask_tensors])
logits = predictions.logits[0]
probabilities = tf.nn.softmax(logits)
predicted_label = tf.argmax(probabilities)

# Print the predicted label and probabilities
if predicted_label == 0:
    print("\n*-*-Fake News-*-*")
else:
    print("\n*-*-Real News-*-*")

print("\nProbability of being fake: {:.2%}".format(probabilities[0]))
print("Probability of being real: {:.2%}".format(probabilities[1]))


Some layers from the model checkpoint at /content/bertv3_model were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/bertv3_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.




Enter News Title: Donald Trump says he has 'good relations with India,' flags 'the only problem'
Preprocessed Text: donald trump say he ha good relat with india flag the onli problem
1/1 [==============================] - 3s 3s/step

*-*-Real News-*-*

Probability of being fake: 0.06%
Probability of being real: 99.94%
